In [108]:
# Carregar o CSV do Brasileirão
df_bra = pd.read_csv('../data/brasileirao/brasileirao_scores.csv', encoding='cp1252')

# Verificar as primeiras linhas para garantir que os dados estão ok
print(df_bra.head())


   k  Day       Date   Time       Home   xG Score xG.1              Away  \
0  1  Sat  3/29/2025  18:30  São Paulo  2.4   0–0  0.4      Sport Recife   
1  1  Sat  3/29/2025  18:30  Juventude  0.7   2–0  0.6           Vitória   
2  1  Sat  3/29/2025  18:30  Fortaleza  0.9   2–0  0.9        Fluminense   
3  1  Sat  3/29/2025  18:30   Cruzeiro  1.2   2–1  1.7          Mirassol   
4  1  Sat  3/29/2025  18:30     Grêmio  0.8   2–1  2.9  Atlético Mineiro   

  Attendance                               Venue  \
0        NaN                  Estádio do Morumbi   
1        NaN              Estádio Alfredo Jaconi   
2        NaN                    Estádio Castelão   
3     40,837  Estádio Governador Magalhães Pinto   
4        NaN                     Arena do Grêmio   

                          Referee  Match Report Notes  
0        Felipe Fernandes de Lima  Match Report   NaN  
1  Paulo Cesar Zanovelli da Silva  Match Report   NaN  
2    Rodrigo José Pereira de Lima  Match Report   NaN  
3     

In [109]:
# Limpar qualquer caractere estranho no placar
df_bra['Score_clean'] = df_bra['Score'].str.replace(r'[^\d\-]', '-', regex=True)

# Verificar a coluna 'Score_clean' para garantir que foi corrigido
print(df_bra[['Home', 'Away', 'Score', 'Score_clean']].head())


        Home              Away Score Score_clean
0  São Paulo      Sport Recife   0–0         0-0
1  Juventude           Vitória   2–0         2-0
2  Fortaleza        Fluminense   2–0         2-0
3   Cruzeiro          Mirassol   2–1         2-1
4     Grêmio  Atlético Mineiro   2–1         2-1


In [110]:
# Substituir os NaN por vazio
df_bra['Score_clean'] = df_bra['Score_clean'].fillna('')

# Verificar o resultado
print(df_bra[['Home', 'Away', 'Score_clean']].head())


        Home              Away Score_clean
0  São Paulo      Sport Recife         0-0
1  Juventude           Vitória         2-0
2  Fortaleza        Fluminense         2-0
3   Cruzeiro          Mirassol         2-1
4     Grêmio  Atlético Mineiro         2-1


In [111]:
# Verificar as linhas com valores não numéricos em 'Score_clean'
df_problemas = df_bra[pd.to_numeric(df_bra['Score_clean'], errors='coerce').isna()]

# Exibir essas linhas para ver o que está errado
print(df_problemas[['Home', 'Away', 'Score_clean']])


                 Home              Away Score_clean
0           São Paulo      Sport Recife         0-0
1           Juventude           Vitória         2-0
2           Fortaleza        Fluminense         2-0
3            Cruzeiro          Mirassol         2-1
4              Grêmio  Atlético Mineiro         2-1
..                ...               ...         ...
424  Atlético Mineiro     Vasco da Gama            
425          Mirassol          Flamengo            
426           Vitória         São Paulo            
427      Sport Recife            Grêmio            
428       Corinthians         Juventude            

[429 rows x 3 columns]


In [112]:
# Remover as linhas com 'NaN' ou valores vazios em 'Score_clean'
df_bra = df_bra[df_bra['Score_clean'].notna()]  # Remover NaN
df_bra = df_bra[df_bra['Score_clean'] != '']  # Remover valores vazios

# Agora, remover qualquer linha onde o placar não tenha o formato correto
df_bra = df_bra[df_bra['Score_clean'].str.match(r'^\d+-\d+$')]

# Verificar as primeiras linhas após a limpeza
print(df_bra[['Home', 'Away', 'Score_clean']].head())


        Home              Away Score_clean
0  São Paulo      Sport Recife         0-0
1  Juventude           Vitória         2-0
2  Fortaleza        Fluminense         2-0
3   Cruzeiro          Mirassol         2-1
4     Grêmio  Atlético Mineiro         2-1


In [113]:
# Dividir a coluna 'Score_clean' em 'Home_Goals_Real' e 'Away_Goals_Real'
df_bra[['Home_Goals_Real', 'Away_Goals_Real']] = df_bra['Score_clean'].str.split('-', expand=True)

# Converter para inteiros
df_bra['Home_Goals_Real'] = df_bra['Home_Goals_Real'].astype(int)
df_bra['Away_Goals_Real'] = df_bra['Away_Goals_Real'].astype(int)

# Verificar os dados
print(df_bra[['Home', 'Away', 'Home_Goals_Real', 'Away_Goals_Real']].head())


        Home              Away  Home_Goals_Real  Away_Goals_Real
0  São Paulo      Sport Recife                0                0
1  Juventude           Vitória                2                0
2  Fortaleza        Fluminense                2                0
3   Cruzeiro          Mirassol                2                1
4     Grêmio  Atlético Mineiro                2                1


In [114]:
# Normalizar os nomes dos times para minúsculas e remover espaços extras
df_bra['Home'] = df_bra['Home'].str.lower().str.strip()
df_bra['Away'] = df_bra['Away'].str.lower().str.strip()
tabela_brasileirao['Squad'] = tabela_brasileirao['Squad'].str.lower().str.strip()

# Adicionar a coluna 'Team_Rank' no DataFrame de jogos com o ranking de cada time
df_bra['Team_Rank'] = df_bra['Home'].map(tabela_brasileirao.set_index('Squad')['Position'])

# Adicionar o ranking do time visitante
df_bra['Away_Team_Rank'] = df_bra['Away'].map(tabela_brasileirao.set_index('Squad')['Position'])

# Verificar se a coluna foi adicionada corretamente
print(df_bra[['Home', 'Away', 'Team_Rank', 'Away_Team_Rank']].head())



        Home              Away  Team_Rank  Away_Team_Rank
0  são paulo      sport recife         16              20
1  juventude           vitória         18              14
2  fortaleza        fluminense         11               5
3   cruzeiro          mirassol          4              12
4     grêmio  atlético mineiro         15               7


In [115]:
# Converter as colunas 'xG' e 'xG.1' para tipo numérico
df_bra['xG'] = pd.to_numeric(df_bra['xG'], errors='coerce')
df_bra['xG.1'] = pd.to_numeric(df_bra['xG.1'], errors='coerce')

# Verificar se a conversão foi feita corretamente
print(df_bra[['Home', 'xG', 'Away', 'xG.1']].head())


        Home   xG              Away  xG.1
0  são paulo  2.4      sport recife   0.4
1  juventude  0.7           vitória   0.6
2  fortaleza  0.9        fluminense   0.9
3   cruzeiro  1.2          mirassol   1.7
4     grêmio  0.8  atlético mineiro   2.9


In [119]:
# Calcular a média de xG do time da casa jogando em casa
df_bra['Home_xG_Avg'] = df_bra.groupby('Home')['xG'].transform('mean')
# Calcular a média de xG do time visitante jogando fora
df_bra['Away_xG_Avg'] = df_bra.groupby('Away')['xG.1'].transform('mean')


# Verificar se a média foi adicionada corretamente
print(df_bra[['Home', 'xG', 'Home_xG_Avg', 'Away', 'xG.1', 'Away_xG_Avg']].head())


        Home   xG  Home_xG_Avg              Away  xG.1  Away_xG_Avg
0  são paulo  2.4        1.575      sport recife   0.4        0.700
1  juventude  0.7        1.175           vitória   0.6        0.800
2  fortaleza  0.9        1.350        fluminense   0.9        0.775
3   cruzeiro  1.2        1.400          mirassol   1.7        1.250
4     grêmio  0.8        0.680  atlético mineiro   2.9        1.650


In [122]:
# Gols feitos pelo time da casa (considerando apenas os gols marcados em casa)
df_bra['Total_Goals_For'] = df_bra.groupby('Home')['Home_Goals_Real'].transform('sum')

# Gols feitos pelo time visitante (considerando apenas os gols marcados fora)
df_bra['Total_Goals_Against'] = df_bra.groupby('Away')['Away_Goals_Real'].transform('sum')

# Verificar os totais de gols feitos e sofridos
print(df_bra[['Home', 'Away', 'Total_Goals_For', 'Total_Goals_Against']].head())


        Home              Away  Total_Goals_For  Total_Goals_Against
0  são paulo      sport recife                3                    3
1  juventude           vitória                6                    3
2  fortaleza        fluminense                8                    4
3   cruzeiro          mirassol                8                    4
4     grêmio  atlético mineiro                5                    4


In [123]:
# Gols sofridos pelo time da casa dentro de casa
df_bra['Home_Goals_Against'] = df_bra.groupby('Home')['Away_Goals_Real'].transform('sum')

# Gols sofridos pelo time visitante fora de casa
df_bra['Away_Goals_Against'] = df_bra.groupby('Away')['Home_Goals_Real'].transform('sum')

# Verificar os totais de gols sofridos
print(df_bra[['Home', 'Away', 'Home_Goals_Against', 'Away_Goals_Against']].head())


        Home              Away  Home_Goals_Against  Away_Goals_Against
0  são paulo      sport recife                   2                   7
1  juventude           vitória                   4                   6
2  fortaleza        fluminense                   2                   7
3   cruzeiro          mirassol                   2                   6
4     grêmio  atlético mineiro                   5                   6


In [145]:
# Inicializando a coluna 'Last_5_Performance' para armazenar os resultados
df_bra['Last_5_Performance'] = ''

# Vamos percorrer os times e calcular os últimos 5 jogos
for team in df_bra['Home'].unique():
    # Filtrando os últimos 5 jogos para o time (como 'Home' ou 'Away')
    team_games = df_bra[(df_bra['Home'] == team) | (df_bra['Away'] == team)].sort_values(by='Date', ascending=False).head(5)
    
    # Determinando os resultados de cada jogo
    performances = []
    for _, row in team_games.iterrows():
        if row['Home'] == team:  # Se o time é o da casa
            if row['Home_Goals_Real'] > row['Away_Goals_Real']:
                performances.append('V')  # Vitória
            elif row['Home_Goals_Real'] == row['Away_Goals_Real']:
                performances.append('E')  # Empate
            else:
                performances.append('D')  # Derrota
        else:  # Se o time é o visitante
            if row['Away_Goals_Real'] > row['Home_Goals_Real']:
                performances.append('V')  # Vitória
            elif row['Away_Goals_Real'] == row['Home_Goals_Real']:
                performances.append('E')  # Empate
            else:
                performances.append('D')  # Derrota

    # Armazenando as últimas 5 performances no DataFrame
    df_bra.loc[df_bra['Home'] == team, 'Last_5_Performance'] = ', '.join(performances)

# Verificar os dados
print(df_bra[['Home', 'Last_5_Performance']].head())


        Home Last_5_Performance
0  são paulo      E, D, E, E, V
1  juventude      D, D, D, D, E
2  fortaleza      E, V, E, E, D
3   cruzeiro      V, V, D, V, D
4     grêmio      V, E, D, E, E


In [149]:
# Contar o número de vitórias nos últimos 5 jogos
df_bra['Last_5_Wins'] = df_bra['Last_5_Performance'].apply(lambda x: x.count('V') if isinstance(x, str) else 0)

# Verificar a nova variável 'Last_5_Wins'
print(df_bra[['Home', 'Last_5_Performance', 'Last_5_Wins']].head())


        Home Last_5_Performance  Last_5_Wins
0  são paulo      E, D, E, E, V            1
1  juventude      D, D, D, D, E            0
2  fortaleza      E, V, E, E, D            1
3   cruzeiro      V, V, D, V, D            3
4     grêmio      V, E, D, E, E            1


In [151]:
from sklearn.linear_model import LinearRegression
import joblib

# Selecionar as variáveis para a previsão
X = df_bra[['Home_xG_Avg', 'Away_xG_Avg', 'Team_Rank', 'Away_Team_Rank', 
            'Total_Goals_For', 'Home_Goals_Against', 'Total_Goals_Against', 
            'Away_Goals_Against', 'Last_5_Wins']]

# O alvo (y) será o número de gols do time da casa (Home_Goals_Real)
y = df_bra['Home_Goals_Real']

# Criar o modelo de regressão linear
modelo = LinearRegression()

# Treinar o modelo
modelo.fit(X, y)

# Salvar o modelo treinado
joblib.dump(modelo, '../models/modelo_brasileirao_com_forma_recenete.pkl')

# Verificar o coeficiente de determinação (R²) para ver o desempenho
r2 = modelo.score(X, y)
print(f"R² : {r2}")


R² : 0.446289546199626


In [154]:
# Prever os gols para os jogos atuais
df_bra['Predicted_Home_Goals'] = modelo.predict(X)

# Verificar as previsões
print(df_bra[['Home', 'Away', 'Home_Goals_Real', 'Predicted_Home_Goals']].head(10))


            Home              Away  Home_Goals_Real  Predicted_Home_Goals
0      são paulo      sport recife                0              1.180397
1      juventude           vitória                2              1.550477
2      fortaleza        fluminense                2              1.931384
3       cruzeiro          mirassol                2              1.678360
4         grêmio  atlético mineiro                2              1.261740
5      palmeiras     botafogo (rj)                0              0.563402
6  vasco da gama            santos                2              1.400710
7          bahia       corinthians                1              1.708162
8       flamengo     internacional                1              3.057821
9  rb bragantino             ceará                2              1.128735
